In [6]:
import random

import matplotlib.pyplot as plt
from task1 import *
from task2 import *
from task3 import *
from task4 import reachability_with_constraints, reachability_all_pairs
import cfpq_data
from scipy.sparse import bsr_matrix, coo_matrix, csr_matrix, csc_matrix, dia_matrix, dok_matrix, lil_matrix
import random
import datetime



In [7]:
graphs = {}
graph_names = ["bzip", "ls", "gzip"]

for graph_name in graph_names:
    graph_path = cfpq_data.download(graph_name)
    graph = cfpq_data.graph_from_csv(graph_path)
    nNodes, nEdges, labels = graph_info(graph_name)
    graphs[graph_name] = {}
    graphs[graph_name]["graph"] = graph
    graphs[graph_name]["nNodes"] = nNodes
    graphs[graph_name]["nEdges"] = nEdges
    graphs[graph_name]["labels"] = labels
    print(graph_name, graph_info(graph_name))
    
graph_queries = {}
graph_queries["bzip"] = ["(a | d)* a", "(a | d)+ d*", "a d a (a|d)*", "(a | d)"]
graph_queries["ls"] = ["(a | d)* a", "(a | d)+ d*", "a d a (a|d)*", "(a | d)"]
graph_queries["gzip"] = ["(a | d)* a", "(a | d)+ d*", "a d a (a|d)*", "(a | d)"]

[2024-03-21 00:46:49]>INFO>Found graph with name='bzip'
[2024-03-21 00:46:50]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-21 00:46:50]>INFO>Unzip graph name='bzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip/bzip.csv')
[2024-03-21 00:46:50]>INFO>Remove archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-21 00:46:50]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x12985cac0> from path=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip/bzip.csv')
[2024-03-21 00:46:50]>INFO>Found graph with name='bzip'
[2024-03-21 00:46:52]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-21 00:46:52]>INFO>Unzip graph name='bzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-pack

bzip (632, 556, {'a', 'd'})


[2024-03-21 00:46:57]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls.tar.gz')
[2024-03-21 00:46:57]>INFO>Unzip graph name='ls' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls/ls.csv')
[2024-03-21 00:46:57]>INFO>Remove archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls.tar.gz')
[2024-03-21 00:46:57]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x12987a130> from path=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls/ls.csv')
[2024-03-21 00:46:57]>INFO>Found graph with name='ls'
[2024-03-21 00:46:59]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls.tar.gz')
[2024-03-21 00:46:59]>INFO>Unzip graph name='ls' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls/ls.csv')
[2024-03-21 00:46:59]>INFO>Remove arc

ls (1687, 1453, {'a', 'd'})


[2024-03-21 00:47:03]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip.tar.gz')
[2024-03-21 00:47:03]>INFO>Unzip graph name='gzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip/gzip.csv')
[2024-03-21 00:47:03]>INFO>Remove archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip.tar.gz')
[2024-03-21 00:47:03]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x1291b2d60> from path=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip/gzip.csv')
[2024-03-21 00:47:03]>INFO>Found graph with name='gzip'
[2024-03-21 00:47:05]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip.tar.gz')
[2024-03-21 00:47:05]>INFO>Unzip graph name='gzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip/gzip.csv')
[2024-03-21 0

gzip (2687, 2293, {'a', 'd'})


In [ ]:
# Исследование влияния типа матрицы на время работы алгоритма

matix_classes_dict = {"csr" : csr_matrix, 
                      "dok" : dok_matrix, 
                      "csc" : csc_matrix, 
                      "lil" : lil_matrix,
                      }
                      

avg_times_reachability_with_constraints_transitive = {}
avg_times_reachability_with_constraints = {}
for matrix_class_name, matrix_class in matix_classes_dict.items():
    for graph_name in graph_names:
        avg_time_reachability_with_constraints_transitive = 0
        avg_time_reachability_with_constraints = 0
        for regex in graph_queries[graph_name]:
            number_of_start_nodes = graphs[graph_name]["nNodes"]//2
            number_of_final_nodes = graphs[graph_name]["nNodes"]//2
            start_states = cfpq_data.generate_multiple_source(graphs[graph_name]["graph"], number_of_start_nodes, seed = 41)
            final_states = cfpq_data.generate_multiple_source(graphs[graph_name]["graph"], number_of_final_nodes, seed = 42)
            graph_nfa = FiniteAutomaton(graph_to_nfa(graphs[graph_name]["graph"], start_states, final_states), matrix_class=matrix_class)
            regex_dfa = FiniteAutomaton(regex_to_dfa(regex), matrix_class=matrix_class)
            
            start = datetime.datetime.now()
            _ = reachability_all_pairs(graph_nfa, regex_dfa, start_states, final_states, matrix_class = matrix_class,matrix_class_id=matrix_class_name)
            finish = datetime.datetime.now()
            avg_time_reachability_with_constraints_transitive += (finish - start).total_seconds()
            
            start = datetime.datetime.now()
            _ = reachability_with_constraints(graph_nfa, regex_dfa, matrix_class = matrix_class,matrix_class_id=matrix_class_name)
            finish = datetime.datetime.now()
            avg_time_reachability_with_constraints += (finish - start).total_seconds()
            
        avg_time_reachability_with_constraints[matrix_class_name] = {}
        avg_time_reachability_with_constraints_transitive[matrix_class_name] = {}
        
        
        avg_time_reachability_with_constraints_transitive /= len(graph_queries[graph_name])
        avg_times_reachability_with_constraints_transitive[matrix_class_name] = avg_time_reachability_with_constraints_transitive
        
        avg_time_reachability_with_constraints /= len(graph_queries[graph_name])
        avg_times_reachability_with_constraints[matrix_class_name] = avg_time_reachability_with_constraints
        
print("Результаты для всех пар вершин: \n")
print(avg_times_reachability_with_constraints_transitive)

print("Результаты для начальных и конечных вершин: \n")
print(avg_times_reachability_with_constraints)
    



[2024-03-21 00:47:13]>INFO>Generate set of source vertices of 316 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x12985cac0> for multiple-source evaluation
[2024-03-21 00:47:13]>INFO>Generate set of source vertices of 316 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x12985cac0> for multiple-source evaluation
/usr/local/lib/python3.9/site-packages/scipy/sparse/_index.py:102: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.9/site-packages/scipy/sparse/_index.py:136: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


Видим, что при вычислении результата для всех пар вершин лучше всего подходит матрица csr, а хуже всего - матрица dok
При вычислении результата для пачки начальных и конечных вершин лучше всего подходит матрица csr, а хуже всего - матрица dok

In [ ]:
# Исследование вопроса о том, когда лучше начинать считать результат при помощи транзитивного замыкания с последующим выбором вершин


all_pairs_times = {}
start_nodes_times = {}
divide = 10

for graph_name in graph_names:
    if not isinstance(all_pairs_times.get(graph_name), dict):
        all_pairs_times[graph_name] = {}
    for regex in graph_queries[graph_name]:
        if not isinstance(start_nodes_times.get(graph_name), dict):
            start_nodes_times[graph_name] = {}
        time_all_pairs = []
        time_start_set = []
        final_states = None
        for number_of_start_nodes in range(0, graphs[graph_name]["nNodes"], graphs[graph_name]["nNodes"]//divide):
            number_of_final_nodes = graphs[graph_name]["nNodes"]//2
            start_states = cfpq_data.generate_multiple_source(graphs[graph_name]["graph"], number_of_start_nodes, seed = 42)
            if final_states is None:
                final_states = cfpq_data.generate_multiple_source(graphs[graph_name]["graph"], number_of_final_nodes, seed = 42)
            
            graph_fa = FiniteAutomaton(graph_to_nfa(graphs[graph_name]["graph"], start_states, final_states))
            regex_fa = FiniteAutomaton(regex_to_dfa(regex))
            
            start1 = datetime.datetime.now()
            _ = reachability_all_pairs(graph_fa, regex_fa, start_states, final_states)
            finish1 = datetime.datetime.now()
            
            start2 = datetime.datetime.now()
            _ = reachability_with_constraints(graph_fa, regex_fa)
            finish2 = datetime.datetime.now()
            
            time_all_pairs.append((finish1 - start1).total_seconds())
            time_start_set.append((finish2 - start2).total_seconds())
            
        all_pairs_times[graph_name][regex] = time_all_pairs
        start_nodes_times[graph_name][regex] = time_start_set


#plot the results
for graph_name in graph_names:
    for regex in graph_queries[graph_name]:
        plt.plot(range(0, graphs[graph_name]["nNodes"], graphs[graph_name]["nNodes"]//divide), all_pairs_times[graph_name][regex], label = "all pairs")
        plt.plot(range(0, graphs[graph_name]["nNodes"], graphs[graph_name]["nNodes"]//divide), start_nodes_times[graph_name][regex], label = "start nodes")
        plt.title(f"Graph {graph_name}, regex {regex}")
        plt.xlabel("Number of start nodes")
        plt.ylabel("Time, s")
        plt.legend()
        plt.show()


Видим, что начиная с числа стартовых вершин около 90 процентов от общего числа вершин в графе нам становится выгоднее запускать алгоритм поиска достижимости между всеми апрами врешин (прри помощи транзитивного замыкания).